In [1]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
df = pd.read_csv('output/TEGR1.csv')

In [3]:
# Inspect number of unique addresses
df.select_dtypes(include=['object']).nunique()

id              253
projectId        16
voter            83
grantAddress     16
dtype: int64

### Applying the QF Algorithm
Inspired by Octopus at https://forum.tecommons.org/t/strengths-and-weaknesses-of-conviction-voting-and-other-mechanisms/1278

```python
def quadratic_fund(str: project) -> float:
    allocation = np.square(np.sum([sqrt(agent.votes.get(project)) for agent in system.agents]))
    return allocation
```

In [4]:
df['sqrt(amountUSD)'] = np.sqrt(df['amountUSD'])
df['sum(sqrt(amountUSD))'] = df.groupby('projectId')['sqrt(amountUSD)'].transform('sum')
df['sq(sum(sqrt(amountUSD)))'] = df['sum(sqrt(amountUSD))'].transform(lambda x: x**2)
df['quadradic_allocation'] = df['sq(sum(sqrt(amountUSD)))'] / df['sq(sum(sqrt(amountUSD)))'].sum()
df['default_allocation'] = df['amountUSD'] / df['amountUSD'].sum()
df['quadratic_amount'] = df['amountUSD'].sum() * df['quadradic_allocation']

In [5]:
df

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag,sqrt(amountUSD),sum(sqrt(amountUSD)),sq(sum(sqrt(amountUSD))),quadradic_allocation,default_allocation,quadratic_amount
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,0.0,3.0,1.0,3.030566,60.250452,3630.117021,0.002959,0.001513,17.961853
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,0.0,0.0,0.0,2.023504,15.075191,227.261396,0.000185,0.000675,1.124492
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,0.0,0.0,0.0,8.628248,56.397388,3180.665361,0.002593,0.012265,15.737962
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0,2.362943,47.809680,2285.765459,0.001863,0.000920,11.309989
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0,2.362943,62.051660,3850.408505,0.003139,0.000920,19.051858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,0.0,0.0,0.0,1.359339,56.397388,3180.665361,0.002593,0.000304,15.737962
249,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0,1.357863,47.809680,2285.765459,0.001863,0.000304,11.309989
250,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0,1.357863,42.547549,1810.293913,0.001476,0.000304,8.957351
251,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,0.0,0.0,0.0,1.357863,7.801686,60.866307,0.000050,0.000304,0.301167


Automating the algorithm.

In [6]:
df = pd.read_csv('output/TEGR1.csv')

In [7]:
def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
    df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
    df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
    df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
    df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
    df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
    df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
    
    return df

In [8]:
qf(df)

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag,amountUSD_allocation,sqrt(amountUSD),sum(sqrt(amountUSD)),sq(sum(sqrt(amountUSD))),quadratic_amount_allocation,quadratic_amount
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,0.0,3.0,1.0,0.001513,3.030566,60.250452,3630.117021,0.002959,17.961853
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,0.0,0.0,0.0,0.000675,2.023504,15.075191,227.261396,0.000185,1.124492
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,0.0,0.0,0.0,0.012265,8.628248,56.397388,3180.665361,0.002593,15.737962
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0,0.000920,2.362943,47.809680,2285.765459,0.001863,11.309989
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0,0.000920,2.362943,62.051660,3850.408505,0.003139,19.051858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,0.0,0.0,0.0,0.000304,1.359339,56.397388,3180.665361,0.002593,15.737962
249,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0,0.000304,1.357863,47.809680,2285.765459,0.001863,11.309989
250,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0,0.000304,1.357863,42.547549,1810.293913,0.001476,8.957351
251,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,0.0,0.0,0.0,0.000304,1.357863,7.801686,60.866307,0.000050,0.301167


In [12]:
# Compute the boosted allocation
df['amount_boosted'] = df['amountUSD'] * df['coefficient']
df = qf(df, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')
df

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,...,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount,amount_boosted,amount_boosted_allocation,quadratic_amount_boosted_allocation,quadratic_amount_boosted,sqrt(amount_boosted),sum(sqrt(amount_boosted)),sq(sum(sqrt(amount_boosted)))
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,...,0.001513,0.002959,17.961853,13.776499,0.001796,0.003273,25.100027,3.711671,70.440067,4961.803089
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,...,0.000675,0.000185,1.124492,4.094567,0.000534,0.000176,1.347561,2.023504,16.321380,266.387447
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,...,0.012265,0.002593,15.737962,74.446665,0.009707,0.002667,20.454132,8.628248,63.587712,4043.397162
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,...,0.000920,0.001863,11.309989,5.583500,0.000728,0.001898,14.554167,2.362943,53.638466,2877.085023
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,...,0.000920,0.003139,19.051858,5.583500,0.000728,0.003276,25.125795,2.362943,70.476216,4966.897020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,...,0.000304,0.002593,15.737962,1.847803,0.000241,0.002667,20.454132,1.359339,63.587712,4043.397162
249,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,...,0.000304,0.001863,11.309989,1.843793,0.000240,0.001898,14.554167,1.357863,53.638466,2877.085023
250,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,...,0.000304,0.001476,8.957351,1.843793,0.000240,0.001338,10.262983,1.357863,45.042187,2028.798632
251,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,...,0.000304,0.000050,0.301167,1.843793,0.000240,0.000046,0.351667,1.357863,8.337738,69.517878


In [11]:
# Remove the intermediate steps
df = df[df.columns[~df.columns.str.contains('sqrt')]]
df

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount,amount_boosted,amount_boosted_allocation,quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,0.0,3.0,1.0,0.001513,0.002959,17.961853,13.776499,0.001796,0.003273,25.100027
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,0.0,0.0,0.0,0.000675,0.000185,1.124492,4.094567,0.000534,0.000176,1.347561
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,0.0,0.0,0.0,0.012265,0.002593,15.737962,74.446665,0.009707,0.002667,20.454132
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0,0.000920,0.001863,11.309989,5.583500,0.000728,0.001898,14.554167
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0,0.000920,0.003139,19.051858,5.583500,0.000728,0.003276,25.125795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,0.0,0.0,0.0,0.000304,0.002593,15.737962,1.847803,0.000241,0.002667,20.454132
249,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0,0.000304,0.001863,11.309989,1.843793,0.000240,0.001898,14.554167
250,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0,0.000304,0.001476,8.957351,1.843793,0.000240,0.001338,10.262983
251,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,0.0,0.0,0.0,0.000304,0.000050,0.301167,1.843793,0.000240,0.000046,0.351667


In [35]:
# Examine the project allocations
allocations = df[['projectId'] + list(df.columns[df.columns.str.contains('allocation')])].groupby('projectId').sum().drop('amount_boosted_allocation',axis=1)
allocations

,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount_boosted_allocation
projectId,,,
0x10b3f00e,0.007163,0.001570,0.001574
0x23387567,0.165409,0.037399,0.043275
0x4cd41869,0.083889,0.177688,0.147632
0x5351510d,0.008092,0.002346,0.002407
0x64a30a4b,0.059451,0.050307,0.055636
0x72b0d6a6,0.011525,0.004762,0.005334
0x8d6f0c7b,0.037947,0.083346,0.090515
0x97589cd1,0.045227,0.049264,0.050672
0xa9bdf738,0.031684,0.078471,0.081902


In [36]:
matching_pool = 25_000
donations = df['amountUSD'].sum()
funding_pool = matching_pool + donations

In [37]:
funding_pool * allocations

,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount_boosted_allocation
projectId,,,
0x10b3f00e,222.540835,48.773026,48.895717
0x23387567,5139.222650,1161.980352,1344.551294
0x4cd41869,2606.410294,5520.706798,4586.873706
0x5351510d,251.426790,72.902157,74.793543
0x64a30a4b,1847.134471,1563.035020,1728.598230
0x72b0d6a6,358.081912,147.958597,165.718247
0x8d6f0c7b,1179.010830,2589.544079,2812.262460
0x97589cd1,1405.188871,1530.631836,1574.365703
0xa9bdf738,984.417323,2438.068922,2544.665969
